<h1 align="center">Changes in PINCER measures during the pandemic</h1>

Pharmacist-led information technology intervention (PINCER) is an effective way to reduce errors in prescription and medication monitoring which are responsible for a large proportion of adverse drug events. This is achieved through the monitoring of 12 PINCER measures.

The purpose of this notebook is observe changes in adherence to each of the PINCER measures at the practice level during the pandemic.  For each of these measures we provide a link to the codelist containing all the codes used for that measure, a description of what the measure is and a brief overview of why the measure is important. A summary of the number of events for each measure is produced and monthly rates of recorded activity for each measure is plotted as a decile chart.

The 12 PINCER measures include:
* ### [Gastrointestinal (GI) Bleed Indicators](#gi_bleed)
    * [Age 65+, on oral NSAID without gastroprotection (GI_P3A)](#a)
    * [H/O peptic ulcer, on oral NSAID without gastroprotection (GI_P3B)](#b)
    * [H/O peptic ulcer, on OAC without gastroprotection (GI_P3C)](#c)
    * [On OAC and oral NSAID (GI_P3D)](#d)
* ### [Monitoring Indicators](#monitoring)
    * [Age 75+, on ACEI or loop, no renal function/electrolytes test (MO_P13)](#ac)
    * [Methotrexate audit (MO_P15)](#me)
        * [On methotrexate without recorded full blood count (FBC)](#me_no_fbc)
        * [On methotrexate without recorded liver functon test (LFT)](#me_no_lft)
    * [On lithium without recent lithium test (MO_P17)](#li)
    * [On amiodarone without recent thyroid function test (TFT) (MO_P18)](#am)
* ### [Other Indicators](#other)
    * [Asthma and non-selective BB (AS_P3G)](#g)
    * [Heart failure and oral NSAID (HF_P3I)](#i)
    * [eGFR <45 and oral NSAID (KI_P3K)](#k)

In [1]:
import pandas as pd
from IPython.display import Image

ModuleNotFoundError: No module named 'cohortextractor'

<a id="gi_bleed"></a>
## Gastrointestinal (GI) Bleed Indicators


<a id="a"></a>
### Age 65+, on oral NSAID without gastroprotection (GI_P3A)

In [2]:
Image(url= "../output/plot_a.jpeg")

<a id="b"></a>
### H/O peptic ulcer, on oral NSAID without gastroprotection (GI_P3B)

In [3]:
Image(url= "../output/plot_b.jpeg")

<a id="c"></a>
### H/O peptic ulcer, on OAC without gastroprotection (GI_P3C)

In [4]:
Image(url= "../output/plot_c.jpeg")

<a id="d"></a>
### On OAC and oral NSAID (GI_P3D)

In [5]:
Image(url= "../output/plot_d.jpeg")

<a id="monitoring"></a>
## Monitoring Indicators

<a id="ac"></a>
### Age 75+, on ACEI or loop, no renal function/electrolytes test (MO_P13)

In [8]:
Image(url= "../output/plot_ac.jpeg")

<a id="me"></a>
### Methotrexate audit (MO_P15)

<a id="me_no_fbc"></a>
#### On methotrexate without recorded full blood count

In [9]:
Image(url= "../output/plot_me_no_fbc.jpeg")

<a id="me_no_lft"></a>
#### On methotrexate without recorded liver function test

In [10]:
Image(url= "../output/plot_me_no_lft.jpeg")

<a id="li"></a>
### On lithium without recent lithium test (MO_P17)

In [11]:
Image(url= "../output/plot_li.jpeg")

<a id="am"></a>
### On amiodarone without recent thyroid function test (MO_P18)

In [12]:
Image(url= "../output/plot_am.jpeg")

<a id="other"></a>
## Other Indicators


<a id="g"></a>
### Asthma and non-selective BB (AS_P3G)

In [13]:
Image(url= "../output/plot_g.jpeg")

<a id="i"></a>
### Heart failure and oral NSAID (HF_P3I)

In [14]:
Image(url= "../output/plot_i.jpeg")

<a id="k"></a>
### eGFR <45 and oral NSAID (KI_P3K)

In [15]:
Image(url= "../output/plot_k.jpeg")